In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import glob
from tqdm import tqdm
import PIL.Image
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import math
from pathlib import Path  
# from d2l import torch as d2l
    
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
l_threshold=10
u_threshold=100
SIZE = 512

In [ ]:
!ls ../working 
data_folder = "/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
train_folder = os.path.join(data_folder, 'train_images')
chain_names = os.listdir(train_folder)

print(os.listdir(data_folder))
print(len(chain_names))

In [ ]:
train_df = pd.DataFrame(columns={'image_id', 'hotel_id'})
for hotel_id in tqdm(chain_names):
    for image_id in os.listdir(os.path.join(train_folder, hotel_id)):
        train_df = train_df.append({'image_id': image_id, 'hotel_id': hotel_id}, ignore_index=True)

In [ ]:
def get_filename(path="/"):
    i=1
    while True:
    
        if str(path)[-i]=="/": 
            break
        i +=1

    #     if i==10: t=False
    return str(path)[-i+1:]
# get_filename(path)

In [ ]:
def change_brightness(img, value=50):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def randomcrop(img, scale=0.5):
    height, width = int(img.shape[0]*scale), int(img.shape[1]*scale)
    x = np.random.randint(0, img.shape[1] - int(width))
    y = np.random.randint(0, img.shape[0] - int(height))
    cropped = img[y:y+height, x:x+width]
    resized = cv2.resize(cropped, (img.shape[1], img.shape[0]))    
    return resized

def noisy(img, noise_type="gauss"):
    if noise_type == "gauss":
        image=img.copy() 
        mean=0
        st=0.7
        gauss = np.random.normal(mean,st,image.shape)
        gauss = gauss.astype('uint8')
        image = cv2.add(image,gauss)
        return image
    elif noise_type == "sp":
        image=img.copy() 
        prob = 0.05
        if len(image.shape) == 2:
            black = 0
            white = 255            
        else:
            colorspace = image.shape[2]
            if colorspace == 3:  # RGB
                black = np.array([0, 0, 0], dtype='uint8')
                white = np.array([255, 255, 255], dtype='uint8')
            else:  # RGBA
                black = np.array([0, 0, 0, 255], dtype='uint8')
                white = np.array([255, 255, 255, 255], dtype='uint8')
        probs = np.random.random(image.shape[:2])
        image[probs < (prob / 2)] = black
        image[probs > 1 - (prob / 2)] = white
        return image

def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img

In [ ]:
def data_augmentation(folder,train_df, number_of_copies=1,dest_folder="../working",u_threshold=100, hotel_id=0):
    if dest_folder=="": dest_folder=folder
    for dirname, _, filenames in os.walk(folder):
        i=0
        if len(filenames)>u_threshold:
            filenames = np.random.choice(filenames, size=u_threshold, replace=False)
            
        for filename in filenames:             
            img = cv2.imread(dirname+"/"+filename,1)
            
            for i in range(number_of_copies):
                transformed = img
                
                if np.random.rand()<0.5:
                    scale = np.random.rand()/2 + 0.5
                    transformed = randomcrop(transformed, scale)
                
                if np.random.rand()<0.5:
                    transformed = cv2.flip(transformed, 1)

                if np.random.rand()<0.5:
                    transformed = cv2.flip(transformed, 0)
                
                if np.random.rand()<0.5:
                    value = np.random.randint(-30, 30) 
                    transformed = change_brightness(transformed, value=value)

                if np.random.rand()<0.3:
                    size = np.random.randint(3,7)
                    transformed = cv2.blur(transformed,(size,size))

                if np.random.rand()<0.2:
                    type_n = np.random.choice(np.array(["gauss","sp"]))
                    transformed = noisy(transformed, type_n)
                
                transformed = pad_image(transformed)
                transformed = cv2.resize(transformed, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
                # if np.random.rand()>0.25:
                    
                name = filename[:-4]+"_"+str(i)+".jpg"
                cv2.imwrite(dest_folder+"/"+name, transformed)
                train_df = train_df.append({'image_id': name, 'hotel_id': hotel_id}, ignore_index=True)
                
            img = pad_image(img)
            img = cv2.resize(img, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
            cv2.imwrite(dest_folder+"/" + filename, img)
    return train_df

In [ ]:
def preprocess_file(path,train_df,dest_folder="../working"):
    print(path)
    img = cv2.imread(path,1)
    
    img= cv2.resize(img, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    filename = get_filename(path)
    cv2.imwrite(dest_folder+"/" + filename, img)
    
    return train_df

In [ ]:

indexes=np.array(train_df["hotel_id"].value_counts().index)
counts = np.array(train_df["hotel_id"].value_counts())

# data_augmentation(os.path.join(train_folder, "10010"), number_of_copies=1,dest_folder="../working")

# train_df[train_df['hotel_id'] == "10010"].index.values
# train_df.values[11017][0]
# path=os.path.join(train_folder, "10010/", str(train_df.values[11017][0]))
# !ls /kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/10010/

# img=cv2.imread(path,1)


# np.random.choice(train_df[train_df['hotel_id'] == indexes[1]].index.values,10)
# os.path.join(train_folder, str(indexes[1]),str(counts[20]))


In [ ]:
for i, index in enumerate(indexes):
    c = counts[i]
  
    if c<l_threshold:
        n_copies = math.floor(l_threshold/c)
        train_df = data_augmentation(os.path.join(train_folder, str(index)),train_df, number_of_copies=n_copies, dest_folder="../working", hotel_id=index)
    else:
        train_df = data_augmentation(os.path.join(train_folder, str(index)),train_df, number_of_copies=0, dest_folder="../working", u_threshold=u_threshold)

In [ ]:
file_list=os.listdir(r"../working")
print(file_list)
reduced_train_df = train_df[train_df['image_id'].isin(file_list)]

In [ ]:
!cd /kaggle/working/ & zip -jqr padded_images_256.zip .
!find . -name "*.jpg" -delete

!ls /kaggle/working

# train_df.to_csv('initial_df.csv')
reduced_train_df.to_csv('final_df.csv')

!ls /kaggle/working
